### Concat AI pieces to add fx, the split


In [16]:
import os
import glob
from pydub import AudioSegment
from pydub.utils import make_chunks 
from math import ceil


slots_durations = {
    'date':      3.0,
    'location':  4.2,
    'user_name': 2.41,
    'car':       5.5
}


def concatWavs(slotName, folderName=None, trimInput=True):
    if folderName == None:
        folderName = slotName

    slot_duration = slots_durations[slotName]
    pathIn   = f'__audio_ai/raw/{folderName}/*breathy*wav' 
    pathOut = f'__audio_ai/raw/{folderName}-raw.wav'
        
    print('concat files from', pathIn, 'with duration', slot_duration)    

    files = glob.glob(pathIn)    
    files.sort()

    combined_sounds = AudioSegment.empty()
    for f in files:
        segment = AudioSegment.from_wav(f)
        
        if trimInput and segment.duration_seconds > slot_duration:
            print('trimmed', f, f"from {segment.duration_seconds} to {slot_duration} secs")
            segment = segment[0:slot_duration*1000]

        silence_duration = ceil(1000 * (slot_duration - segment.duration_seconds))
        if silence_duration < 0:
            print('[ERROR]', f, f"file_duration ({segment.duration_seconds}) is longer than slot_duration ({slot_duration})")
            continue        
        
        segment += AudioSegment.silent(duration=silence_duration)
        chunks = make_chunks(segment, slot_duration*1000)
        combined_sounds += chunks[0]

        print('added', f, end='                                                      \r') 

    combined_sounds.export(pathOut, format="wav")    
    print('concat complete', pathOut)


def splitWav(slotName, folderName=None):
    if folderName == None:
        folderName = slotName

    fileNames = glob.glob(f'__audio_ai/raw/{folderName}/*breathy*wav')
    fileNames.sort()

    slot_duration = slots_durations[slotName]
    pathIn = f'__audio_ai/processed/{folderName}-raw-wFilter.wav'    
    dirOut = f'__audio_ai/processed/chunks/{folderName}/' 

    os.makedirs(dirOut, exist_ok=True)
    
    print('split file', pathIn, 'to', dirOut, 'with duration', slot_duration)    
    
    combined = AudioSegment.from_file(pathIn, "wav") 
    chunks = make_chunks(combined, slot_duration*1000)

    for i, chunk in enumerate(chunks): 
        fileName = os.path.basename(fileNames[i]).replace(f'{slotName}-', '', 1).replace('_spk_breathy_syn', '', 1).replace('-', '_')
        chunk.export(dirOut + fileName, format="wav") 
    
    print(len(chunks), 'chunks created')

In [ ]:
concatWavs('car', folderName='car-2ndBatch')

In [ ]:
splitWav('car', folderName='car-1stBatch')

### Shift user names to add a silence at the begining

In [12]:
import os
import glob
from pydub import AudioSegment
from pydub.utils import make_chunks 
from math import ceil


slot_duration = 2.41
lead_silence = 446 # in millis

dirOut = '_audio_ai/processed/chunks/user_name_(with_silence)/'


def shiftSilence(pathIn, dirOut):    
    # add silence
    audio = AudioSegment.silent(duration=lead_silence) 
    audio += AudioSegment.from_file(pathIn, "wav") 
    # trim
    chunks = make_chunks(audio, slot_duration*1000) 
    audio = chunks[0]

    fileName = os.path.basename(pathIn)
    audio.export(dirOut + fileName, format="wav") 


os.makedirs('_audio_ai/processed/chunks/user_name_(with_silence)/', exist_ok=True)

fileNames = glob.glob('_audio_ai/processed/chunks/user_name/*.wav')
fileNames.sort()

for f in fileNames:
    shiftSilence(f, dirOut)
